In [1]:
### Bounding box om letters heen, daarna herkennen
#->

# andere dataset om letters te trainen, daarna dat model toepassen op mijn dataset

!pip3 install protobuf==3.20.3
!pip3 install tensorflow-datasets keras-tuner

import numpy as np
import os
import PIL
import PIL.Image
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import tensorflow_datasets as tfds
from kerastuner.tuners import RandomSearch

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

/tmp/ipykernel_1937/1140250797.py:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
import pathlib
data_dir = "./dataset/ocr_characters/"
#archive = tf.keras.utils.get_file(origin=dataset_url)
#data_dir = pathlib.Path(archive).with_suffix('')

In [8]:
batch_size = 16
img_h = 120
img_w = 120

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir + 'training_data/',
    validation_split=0.2,
    subset="training",
    seed=42,
    shuffle=True,
    image_size=(img_h, img_w),
    batch_size=batch_size)

Found 20628 files belonging to 36 classes.
Using 16503 files for training.


In [9]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir + 'testing_data/',
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_h, img_w),
  batch_size=batch_size)

class_names = train_ds.class_names

Found 1008 files belonging to 36 classes.
Using 201 files for validation.


In [10]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Rescaling(1./255))
    
    hp_filters = hp.Int('filters', min_value=16, max_value=64, step=16)
    model.add(tf.keras.layers.Conv2D(hp_filters, 3, activation='relu'))
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(tf.keras.layers.Conv2D(hp.Int(f'filters_{i}', min_value=16, max_value=64, step=16),
                            kernel_size=hp.Choice(f'kernel_size_{i}', values=[3, 5]), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    
    model.add(tf.keras.layers.Flatten())
    
    hp_units = hp.Int('units', min_value=32, max_value=64, step=32)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    
    model.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10)

tuner.search(train_ds, validation_data=val_ds, epochs=5)

best_model = tuner.get_best_modes(num_models=1)[0]
history = best_model.evaluate(val_ds)

INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
48                |48                |filters
2                 |2                 |num_layers
16                |16                |filters_0
5                 |5                 |kernel_size_0
32                |32                |units
0.0001            |0.0001            |learning_rate
32                |32                |filters_1
3                 |3                 |kernel_size_1

Epoch 1/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/hypermodel.py", line 144, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.wi

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/hypermodel.py", line 144, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/ke

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

result = model.evaluate(val_ds)
dict(zip(model.metrics_names, result))